In [66]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd

In [10]:
opts=webdriver.ChromeOptions()
opts.headless=True
driver = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)

driver.get("https://pergikuliner.com/restaurants/jakarta/bakmi-bintang-gading-kelapa-gading")

content = driver.page_source
soup = BeautifulSoup(content)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
 
[WDM] - Driver [/home/rudi/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


In [87]:
# revbox = soup.findAll('div', attrs={"class": "review-wrapper"})
revbox = soup.find('div', attrs={"class": "review-wrapper"})

In [53]:
# user_information
usr = revbox.find('div', attrs={"class": "usr-info"})
username = usr.find("span", attrs={"itemprop": "name"}).text
review_count = usr.div.findAll('p')[-2].text.split()[0]
user_level = usr.div.findAll('p')[-1].text.split()[1]

In [36]:
# review
review = revbox.find('div', attrs={'class':'top-part'}).p.text
review

'salah satu bakmi favorit di kelapa gading, order via grabfood karena ada promo dan enaknya ga kalah dari dine in. suka banget sama tekstur mie kecilnya, lembut tipis tipis dan kenyal pas. aroma bakmi nya juga wangi daging nya empuk, packaging nya juga ok aman semua dipisah2 jadi lebih higienis. highly recommended!!'

In [48]:
tanggal = revbox.find("div", attrs={'class': 'bottom-part'}).findAll('span')[0].text.split(":")[1]

In [54]:
print(re.sub("\s+", " ",username), review_count, user_level, review, tanggal)

 Fioo | @eatingforlyfe  213 11 salah satu bakmi favorit di kelapa gading, order via grabfood karena ada promo dan enaknya ga kalah dari dine in. suka banget sama tekstur mie kecilnya, lembut tipis tipis dan kenyal pas. aroma bakmi nya juga wangi daging nya empuk, packaging nya juga ok aman semua dipisah2 jadi lebih higienis. highly recommended!!  03 November 2020


In [89]:
def get_reviews(url_post,max_iter = 1000):
    driver.get(url_post)
    content = driver.page_source
    soup = BeautifulSoup(content)
    usernames = []
    review_counts = []
    user_levels = []
    reviews = []
    datetime = []
    for y in range(max_iter):
        revbox = soup.findAll('div', attrs={"class": "review-wrapper"})
        iterate = 0
        for x in revbox:
            usr = x.find('div', attrs={"class": "usr-info"})
            username = re.sub("\s+"," ",usr.find("span", attrs={"itemprop": "name"}).text)
            review_count = usr.div.findAll('p')[-2].text.split()[0]
            user_level = usr.div.findAll('p')[-1].text.split()[1]
            review = x.find('div', attrs={'class':'top-part'}).p.text
            tanggal = x.find("div", attrs={"class": "bottom-part"}).findAll("span")[-2].text.split(":")[1]
            usernames.append(username) 
            review_counts.append(review_count)
            user_levels.append(user_level)
            datetime.append(tanggal)
            reviews.append(review)

        button = soup.find("div", attrs={"id": "load-review-container"})
        if button.find('a'):
            driver.get(url_post + "?page=" + str(y+1))
            content = driver.page_source
            soup = BeautifulSoup(content)
        else:
            break

    return {
        "username": usernames,
        "review_count": review_counts,
        "user_level": user_levels,
        "datetime": datetime,
        "review": reviews
    }

pd.DataFrame(get_reviews("https://pergikuliner.com/restaurants/jakarta/bakmi-bintang-gading-kelapa-gading"))

,username,review_count,user_level,datetime,review
0,Fioo | @eatingforlyfe,213,11,03 November 2020,"salah satu bakmi favorit di kelapa gading, ord..."
1,@eatfoodtravel,416,12,29 Oktober 2020,Ordered mie ujung pandang with some babi garin...
2,vio kal,203,12,23 Juni 2020,Balik lagi makan di @bakmibintanggadingofficia...
3,Makan2 TV Food & Travel,657,15,06 Oktober 2019,Bakmi Bintang Gading merupakan salah satu temp...
4,Julian with danisa,150,9,30 Agustus 2019,Siang ini tiba tiba kepengen makan Bakmi Binta...
5,Indra Mulia,2151,19,30 Maret 2019,Sabtu pagi sarapan sama makanan khas Makassar ...
6,inggie @makandll,431,13,13 Oktober 2018,"Enak, porsi sangat mengenyangkan.Tempat ok, se..."
7,IG: waktukumakan,13,3,10 September 2018,Kali ini kita cicipi makanan khas Makassar du...
8,Andri,210,11,28 Juli 2018,Ini restoran udah lama banget ada dikelapa ga...
9,Fransiscus,1139,17,26 Mei 2018,Mencicipi salah satu resep bakmie khas Makassa...


In [90]:
hangtuah_kopi = pd.DataFrame(get_reviews("https://pergikuliner.com/restaurants/jakarta/hang-tuah-kopi-dan-toastery-kelapa-gading-2"))

In [92]:
hangtuah_kopi.to_csv("hangtuah_kopi.csv")